# This is the notebook for Coursera Applied Data Science Capstone Project

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## 1- Introduction <a name="introduction"></a>

Open a new business can be a challenge. Will it be a sucess or will it fail? What kind of business I can open here? Where is the best place to open this kind of business?

This are some questions that motivated this project. With some data and knwoledge, we will try to awnser some of this questions.

This project uses **Foursquare Database** to find Toronto's nearby restaurants and its rates trying to understand where is the best place to open a new restaurant and what cuisine to focus.

## 2- Data <a name="data"></a>

For this example, I will use Toronto as baseline, but it can be use with any city that exist in Foursquare database.

I will use more venues' informations than previous projects, such as price, popularity, if its clean or if its require any dress code. 

Following data sources will be needed to extract/generate the required information:
* centers of areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* information about restaurants as their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Toronto's neighborhoods will be obtained using **Google Maps API geocoding**

### Finding all neighborhoods

For this part we will find all neighborhoods in Toronto and it's latitude and longitude coordinates.

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# GET HTML INFORMATION
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url).text
soup=BeautifulSoup(html_data,'html5lib')

# CREATE DATAFRAME
columns = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = columns)

# WEBSCRAPPING INFORMATION TO FILL THE DATAFRAME
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                     'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                     'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                     'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


**For latitude and longitude we will use the course CSV file because, as we saw during the course, geocoder is kind of unreliable and may not work well.**

In [4]:
geo_df=pd.read_csv('Geospatial_Coordinates.csv')
geo_df=geo_df.rename(columns={"Postal Code": "PostalCode"})
geo_df.sort_values(by = ['PostalCode'], inplace=True)
df.sort_values(by = ['PostalCode'], inplace=True)
toronto_df = pd.merge(df, geo_df, on='PostalCode')
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [5]:
pip install geopy folium

Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [7]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [8]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [9]:
CLIENT_ID = 'K1NJ4WQH0IQ1M2RQ3F3AUUYWK2H5DUBH2IPIGYB4FQ3Y2GPK' # your Foursquare ID
CLIENT_SECRET = 'J1K0L5M5J22KM403ZQOGLIV5R2ZGQBFRRJUGFRISQCJURHVZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Credentials Saved')

Credentials Saved


**We will have to work with reduced number of neighborhood because there is limitations on API calls**

In [11]:
print(toronto_df['Borough'].unique())

['Scarborough' 'North York' 'East York' 'East Toronto'
 'East York/East Toronto' 'Central Toronto' 'Downtown Toronto'
 'Downtown Toronto Stn A' 'York' 'West Toronto' "Queen's Park"
 'Mississauga' 'East Toronto Business' 'Etobicoke' 'Etobicoke Northwest']


**We will work with all neighborhoods with borough that contains York on its name**

In [85]:
neighbor_df = toronto_df[toronto_df['Borough'].str.contains('York')].reset_index(drop=True)
neighbor_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714
4,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493
5,M2N,North York,Willowdale South,43.770120,-79.408493
6,M2P,North York,York Mills West,43.752758,-79.400049
7,M2R,North York,Willowdale West,43.782736,-79.442259
8,M3A,North York,Parkwoods,43.753259,-79.329656
9,M3B,North York,Don Mills North,43.745906,-79.352188


In [86]:
map_toronto_nb = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighbor_df['Latitude'], neighbor_df['Longitude'], neighbor_df['Borough'], neighbor_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_nb)  
    
map_toronto_nb

In [87]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue ID',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [88]:
tor_nb_venues = getNearbyVenues(names=neighbor_df['Neighborhood'],
                                   latitudes=neighbor_df['Latitude'],
                                   longitudes=neighbor_df['Longitude']
                                  )

**Here we retrieve all Venues existing in all neighborhoods**

In [89]:
print(tor_nb_venues.shape)
tor_nb_venues.head()

(332, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,4d8771f6651041bd5e499b30,New York Fries,43.803664,-79.363905,Fast Food Restaurant
1,Hillcrest Village,43.803762,-79.363452,4ad9dce6f964a520651b21e3,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
2,Hillcrest Village,43.803762,-79.363452,4e1fad69fa761d67106a9989,AY Jackson Pool,43.804515,-79.366138,Pool
3,Hillcrest Village,43.803762,-79.363452,5331a799498e3bfbae4a5a04,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
4,Hillcrest Village,43.803762,-79.363452,4c64ca1b772bef3bfadbc2aa,Duncan Creek Park,43.805539,-79.360695,Dog Run


In [83]:
print(tor_nb_venues["Venue Category"].unique())

['Playground' 'Park' 'Trail' 'Diner' 'Italian Restaurant'
 'Japanese Restaurant' 'Restaurant' 'Café' 'Indian Restaurant' 'Bakery'
 'Jewelry Store' 'Butcher' 'General Entertainment' 'Gastropub'
 'Deli / Bodega' 'Pet Store' 'Caribbean Restaurant' 'Pub'
 'Taiwanese Restaurant' 'Gift Shop' 'Coffee Shop' 'Market' 'Bank'
 'Thai Restaurant' 'Liquor Store' 'Beer Store' 'Sandwich Place'
 'Pizza Place' 'Chinese Restaurant' 'Grocery Store' 'Pharmacy'
 'Snack Place' 'Theme Restaurant' 'Dance Studio' 'Bubble Tea Shop'
 'Breakfast Spot' 'Beer Bar' 'Bookstore' 'Ramen Restaurant'
 'Mexican Restaurant' 'Salon / Barbershop' 'Juice Bar' 'Creperie'
 'Martial Arts School' 'Burger Joint' 'Gay Bar' 'Sushi Restaurant'
 'Escape Room' 'Burrito Place' 'Steakhouse' "Men's Store" 'Smoke Shop'
 'Yoga Studio' 'Hotel' 'Sake Bar' 'Dog Run' 'Theater'
 'Health & Beauty Service' 'American Restaurant'
 'Mediterranean Restaurant' 'Clothing Store' 'Fast Food Restaurant'
 'Food & Drink Shop' 'Strip Club' 'Sculpture Garden' '

**Now we filter all Venues that is categorized as Restaurant, although there are other categories that works with food**

In [95]:
filtered_venues = tor_nb_venues[tor_nb_venues["Venue Category"].str.contains('Restaurant')]
filtered_venues.shape
filtered_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,4d8771f6651041bd5e499b30,New York Fries,43.803664,-79.363905,Fast Food Restaurant
3,Hillcrest Village,43.803762,-79.363452,5331a799498e3bfbae4a5a04,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
7,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4ae083d4f964a520d47f21e3,Moxie's Classic Grill,43.777779,-79.343185,American Restaurant
18,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4af5ff72f964a520f6ff21e3,New York Fries - Fairview Mall,43.778605,-79.343577,Restaurant
25,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,5952e5ed491be7022cf5d19f,Heart Sushi,43.777203,-79.343805,Japanese Restaurant
29,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,5650fa8e498e405c38a5e6d4,Thai Express,43.777990,-79.344091,Restaurant
38,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4c8ff7f5de3d236a27d86718,KFC,43.777600,-79.344200,Fast Food Restaurant
39,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4ae1aee0f964a520be8621e3,Spring Rolls,43.777351,-79.343977,Asian Restaurant
43,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4c16833b96040f4707f672a5,Bourbon St. Grill,43.778276,-79.343241,Fast Food Restaurant
44,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4c7ab7d6a8683704868a144d,McDonald's,43.778407,-79.343574,Fast Food Restaurant


In [108]:
def getVenuesInfo(name, venue_id):
    
    venues_list=[]
    for name, venue in zip(name, venue_id): 
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
        # make the GET request
        results = requests.get(url).json()["response"]['venue']
        
        # return only relevant information for each nearby venue
        rating = 0
        tips = 0
        price = 0
        if 'rating' in results:
            rating = results['rating']
        if 'tips' in results:
            if 'count' in results['tips']:
                tips = results['tips']['count']
        if 'price' in results:
            if 'count' in results['price']:
                price = results['price']['tier']
        venues_list.append([(
            name,
            results['id'],
            results['name'], 
            results['location']['lat'], 
            results['location']['lng'],  
            results['categories'][0]['name'],
            rating,
            tips,
            price,
            results['likes']['count'])
            ])
    return venues_list

In [109]:
venue_list = getVenuesInfo(name=filtered_venues['Neighborhood'] ,venue_id=filtered_venues['Venue ID'])

In [183]:
nearby_venues = pd.DataFrame([item for v_list in venue_list for item in v_list])
nearby_venues.columns = [
                  'Neighborhood',
                  'Venue ID',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue Rating',
                  'Venue Total Tips',
                  'Venue Price Rate',
                  'Venue Total Likes']
nearby_venues

,Neighborhood,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Rating,Venue Total Tips,Venue Price Rate,Venue Total Likes
0,Hillcrest Village,4d8771f6651041bd5e499b30,New York Fries,43.803664,-79.363905,Fast Food Restaurant,6.4,0,0,1
1,Hillcrest Village,5331a799498e3bfbae4a5a04,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant,0.0,0,0,0
2,"Fairview, Henry Farm, Oriole",4ae083d4f964a520d47f21e3,Moxie's Classic Grill,43.777779,-79.343185,American Restaurant,7.8,56,0,75
3,"Fairview, Henry Farm, Oriole",4af5ff72f964a520f6ff21e3,New York Fries - Fairview Mall,43.778605,-79.343577,Restaurant,7.0,2,0,7
4,"Fairview, Henry Farm, Oriole",5952e5ed491be7022cf5d19f,Heart Sushi,43.777203,-79.343805,Japanese Restaurant,6.6,3,0,5
5,"Fairview, Henry Farm, Oriole",5650fa8e498e405c38a5e6d4,Thai Express,43.777990,-79.344091,Restaurant,6.3,0,0,0
6,"Fairview, Henry Farm, Oriole",4c8ff7f5de3d236a27d86718,KFC,43.777600,-79.344200,Fast Food Restaurant,6.0,1,0,2
7,"Fairview, Henry Farm, Oriole",4ae1aee0f964a520be8621e3,Spring Rolls,43.777351,-79.343977,Asian Restaurant,6.0,28,0,30
8,"Fairview, Henry Farm, Oriole",4c16833b96040f4707f672a5,Bourbon St. Grill,43.778276,-79.343241,Fast Food Restaurant,5.9,2,0,6
9,"Fairview, Henry Farm, Oriole",4c7ab7d6a8683704868a144d,McDonald's,43.778407,-79.343574,Fast Food Restaurant,5.7,8,0,9


In [184]:
# All values equals to 0
nearby_venues.drop(columns='Venue Price Rate', inplace=True)
nearby_venues

,Neighborhood,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Rating,Venue Total Tips,Venue Total Likes
0,Hillcrest Village,4d8771f6651041bd5e499b30,New York Fries,43.803664,-79.363905,Fast Food Restaurant,6.4,0,1
1,Hillcrest Village,5331a799498e3bfbae4a5a04,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant,0.0,0,0
2,"Fairview, Henry Farm, Oriole",4ae083d4f964a520d47f21e3,Moxie's Classic Grill,43.777779,-79.343185,American Restaurant,7.8,56,75
3,"Fairview, Henry Farm, Oriole",4af5ff72f964a520f6ff21e3,New York Fries - Fairview Mall,43.778605,-79.343577,Restaurant,7.0,2,7
4,"Fairview, Henry Farm, Oriole",5952e5ed491be7022cf5d19f,Heart Sushi,43.777203,-79.343805,Japanese Restaurant,6.6,3,5
5,"Fairview, Henry Farm, Oriole",5650fa8e498e405c38a5e6d4,Thai Express,43.777990,-79.344091,Restaurant,6.3,0,0
6,"Fairview, Henry Farm, Oriole",4c8ff7f5de3d236a27d86718,KFC,43.777600,-79.344200,Fast Food Restaurant,6.0,1,2
7,"Fairview, Henry Farm, Oriole",4ae1aee0f964a520be8621e3,Spring Rolls,43.777351,-79.343977,Asian Restaurant,6.0,28,30
8,"Fairview, Henry Farm, Oriole",4c16833b96040f4707f672a5,Bourbon St. Grill,43.778276,-79.343241,Fast Food Restaurant,5.9,2,6
9,"Fairview, Henry Farm, Oriole",4c7ab7d6a8683704868a144d,McDonald's,43.778407,-79.343574,Fast Food Restaurant,5.7,8,9


In [135]:
nearby_venues.groupby('Neighborhood').count()

,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Rating,Venue Total Tips,Venue Total Likes
Neighborhood,,,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",4,4,4,4,4,4,4,4
Bayview Village,2,2,2,2,2,2,2,2
"Bedford Park, Lawrence Manor East",10,10,10,10,10,10,10,10
"Del Ray, Mount Dennis, Keelsdale and Silverthorn",2,2,2,2,2,2,2,2
Don Mills North,2,2,2,2,2,2,2,2
Don Mills South,6,6,6,6,6,6,6,6
"Fairview, Henry Farm, Oriole",11,11,11,11,11,11,11,11
Glencairn,2,2,2,2,2,2,2,2
Hillcrest Village,2,2,2,2,2,2,2,2


In [149]:
print(nearby_venues['Neighborhood'].unique().shape)

(15,)


## Methodology <a name="methodology"></a>

For this project, our focus is to indentify each restaurant and cluster then in categories, identify it's location and show the occurency of each for each neighborhood and show its rating to try to understand each neighborhood preferency.

In [154]:
# one hot encoding
venues_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_onehot['Neighborhood'] = nearby_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Dim Sum Restaurant,Fast Food Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Sushi Restaurant,Thai Restaurant,Vietnamese Restaurant
0,Hillcrest Village,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,"Fairview, Henry Farm, Oriole",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [178]:
venue_prop = nearby_venues[['Venue Rating', 'Venue Total Tips', 'Venue Total Likes']]
# kmeans_venues = pd.concat([venues_onehot, venue_prop], axis=1)
kmeans_venues = venues_onehot
kmeans_clustering = kmeans_venues.drop(columns='Neighborhood')

In [179]:
from sklearn.cluster import KMeans

number_of_clusters = 15
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(kmeans_clustering)


In [182]:
kmeans_venues.insert(0, 'Cluster Labels', kmeans.labels_)
kmeans_venues

,Cluster Labels,Neighborhood,American Restaurant,Asian Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Dim Sum Restaurant,Fast Food Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Sushi Restaurant,Thai Restaurant,Vietnamese Restaurant
0,1,Hillcrest Village,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,10,"Fairview, Henry Farm, Oriole",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,2,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
5,3,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
6,1,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,11,"Fairview, Henry Farm, Oriole",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [198]:
venues_merged = pd.concat([nearby_venues,kmeans_venues], axis=1)
venues_merged = venues_merged.loc[:,~venues_merged.columns.duplicated()]
venues_merged

,Neighborhood,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Rating,Venue Total Tips,Venue Total Likes,Cluster Labels,American Restaurant,Asian Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Dim Sum Restaurant,Fast Food Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Sushi Restaurant,Thai Restaurant,Vietnamese Restaurant
0,Hillcrest Village,4d8771f6651041bd5e499b30,New York Fries,43.803664,-79.363905,Fast Food Restaurant,6.4,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,5331a799498e3bfbae4a5a04,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,"Fairview, Henry Farm, Oriole",4ae083d4f964a520d47f21e3,Moxie's Classic Grill,43.777779,-79.343185,American Restaurant,7.8,56,75,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Fairview, Henry Farm, Oriole",4af5ff72f964a520f6ff21e3,New York Fries - Fairview Mall,43.778605,-79.343577,Restaurant,7.0,2,7,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,"Fairview, Henry Farm, Oriole",5952e5ed491be7022cf5d19f,Heart Sushi,43.777203,-79.343805,Japanese Restaurant,6.6,3,5,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
5,"Fairview, Henry Farm, Oriole",5650fa8e498e405c38a5e6d4,Thai Express,43.777990,-79.344091,Restaurant,6.3,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
6,"Fairview, Henry Farm, Oriole",4c8ff7f5de3d236a27d86718,KFC,43.777600,-79.344200,Fast Food Restaurant,6.0,1,2,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,"Fairview, Henry Farm, Oriole",4ae1aee0f964a520be8621e3,Spring Rolls,43.777351,-79.343977,Asian Restaurant,6.0,28,30,11,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,"Fairview, Henry Farm, Oriole",4c16833b96040f4707f672a5,Bourbon St. Grill,43.778276,-79.343241,Fast Food Restaurant,5.9,2,6,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9,"Fairview, Henry Farm, Oriole",4c7ab7d6a8683704868a144d,McDonald's,43.778407,-79.343574,Fast Food Restaurant,5.7,8,9,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [212]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(number_of_clusters)
ys = [i + x + (i*x)**2 for i in range(number_of_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cat, rate, cluster in zip(venues_merged['Venue Latitude'], venues_merged['Venue Longitude'], venues_merged['Venue'],venues_merged['Venue Category'],venues_merged['Venue Rating'], venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + '/' + str(cat) + ' Rating: ' + str(rate) + ' Cluster ' + str(cluster), parse_html=True)
    number = int(cluster)-1
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[number],
        fill=True,
        fill_color=rainbow[number],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Results and Discussion <a name="results"></a>

The study shows that there are concentration of some types of restaurants. There is some density in restaurants is some streets. 

One tendency shown on this study is that, where there are similar restaurants, the rating of each usally is lower than where there are only one type of each cuisine. With this we can conclude that diversity in cuisine types benefits all restaurants around some area.

As a side effect of the study, we found areas which are lacking restaurant services, which can be good places to start some fresh new, without competition.

## Conclusion <a name="conclusion"></a>

The purpose of this study was to help new business deciding where and what to do. Although we face some adversities with lack of data, or API related problems, this study gave us some usefull information. By analizing the distribution of some service we could find some tendency and correlations that can help new restaurants.

This study was the first idea in this field and it should be used as an example. The main idea is that we can use this core concepts to work with any type of business, not only restaurants, and anywhere. With good database